# LSTM Model

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
xtrain_data_path = "/home/panza/Downloads/UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals/"
ytrain_data_path = "/home/panza/Downloads/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"
xtest_data_path = "/home/panza/Downloads/UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals/"
ytest_data_path = "/home/panza/Downloads/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"
data_groups=["body_acc", "total_acc", "body_gyro"]

## Load Dataset
Creo que lo difícil de esto va a ser transformar el dataset de campañas en algo similar al dataset de HAR

In [3]:
def load_file(path):
    """load files"""
    np_array = pd.read_csv(path,
                          header=None,
                          delim_whitespace=True)\
                 .values
    # con .values, devuelvo un array, que es lo que voy a necesitar para trabajar
    
    return np_array


def load_groups(path, groups=["main"]):
    """Load all groups into a dictionary
    if no_groups ---> sigle group named "main"
    """
    no_groups = False
    file_list = os.listdir(path)
    df_dict = {}
    if len(groups) == 1:
        no_groups = True
    
    for group in groups:
        array_list = []
        for file in file_list:
            if (group in file) or (no_groups):
                print(file[:-4], "loaded")  # [:-4 para sacarle el .txt]
                array_list.append(load_file(path + file))

        df_dict[group] = np.dstack(array_list)
        print(group, "group loaded!\n")
        
    print("\n\nData Loaded!\n")
    
    return df_dict


def concat_groups(array_dict, axis=2):
    """Takes every array in the dict and concatenates them accross an axis"""
    array_list = list(array_dict.values())
    return np.concatenate(array_list, axis=axis)

In [4]:
X_train_array_dict = load_groups(xtrain_data_path, data_groups)
y_train_array = load_file(ytrain_data_path)

X_test_array_dict = load_groups(xtest_data_path, data_groups)
y_test_array = load_file(ytest_data_path)

body_acc_z_train loaded
body_acc_x_train loaded
body_acc_y_train loaded
body_acc group loaded!

total_acc_y_train loaded
total_acc_z_train loaded
total_acc_x_train loaded
total_acc group loaded!

body_gyro_x_train loaded
body_gyro_y_train loaded
body_gyro_z_train loaded
body_gyro group loaded!



Data Loaded!

body_acc_z_test loaded
body_acc_y_test loaded
body_acc_x_test loaded
body_acc group loaded!

total_acc_x_test loaded
total_acc_z_test loaded
total_acc_y_test loaded
total_acc group loaded!

body_gyro_x_test loaded
body_gyro_z_test loaded
body_gyro_y_test loaded
body_gyro group loaded!



Data Loaded!



In [5]:
X_train_array_dict["body_acc"].shape

(7352, 128, 3)

#### Hasta acá, tengo un NumPy array
#### (7352, 128) significa que tengo 7352 windows de datos, y cada window tiene 128 observaciones
#### (7352, 128, 3) significa lo mismo, pero en 3 dimensiones. 7352 windows, 128 obs por window, 3 dimensiones (x, y, z)

In [6]:
print(y_train_array.shape)

(7352, 1)


In [7]:
X_test_array_dict["body_acc"].shape

(2947, 128, 3)

#### Para hacerlo coincidir con el libro, me armo una función que me concatene los 3 grupos...

In [8]:
X_train_array = concat_groups(X_train_array_dict)
X_test_array = concat_groups(X_test_array_dict)

In [9]:
print(X_train_array.shape, X_test_array.shape)
print(y_train_array.shape, y_test_array.shape)

(7352, 128, 9) (2947, 128, 9)
(7352, 1) (2947, 1)


## Balance of Activity Classes. 

In [10]:
pd.DataFrame(y_train_array).groupby(0).size()

0
1    1226
2    1073
3     986
4    1286
5    1374
6    1407
dtype: int64

### En este caso, están balanceadas. Pero en el caso de la soja, voy a tener que balancear el dataset. 
(Me voy a tener que armar un custom dataset)

## Problem Modeling

Una verga el capitulo... puro relleno